In [ ]:
# importing libraries
!pip install scikit-learn xgboost joblib
import numpy as np
import matplotlib.pyplot as mtp
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import urllib.request

In [ ]:
try:
    # Try loading locally
    df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
    print("Loaded dataset from local file.")

except FileNotFoundError:
    print("Local dataset not found. Attempting to download...")

    try:
        url = "https://raw.githubusercontent.com/SohelRaja/Customer-Churn-Analysis/master/Datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv"
        urllib.request.urlretrieve(url, "WA_Fn-UseC_-Telco-Customer-Churn.csv")

        df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
        print("Dataset downloaded successfully!")

    except Exception as e:
        print("Error downloading dataset:", str(e))
        print("Please download manually from Kaggle:")
        print("https://www.kaggle.com/datasets/blastchar/telco-customer-churn")
        df = None

if df is not None:
    print("Dataset preview:")
    display(df.head())

Local dataset not found. Attempting to download...
Dataset downloaded successfully!
Dataset preview:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
